In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import mode
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import warnings

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)




In [3]:

# Load dataset
df = pd.read_csv("./dataset/heart.csv")

In [4]:

# Encode categorical features
label_encoder = LabelEncoder()
categorical_cols = ['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

# Split the data into features (X) and target variable (y)
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Load pretrained models
ann_model = load_model('./models/ANN.h5')
gru_model = load_model('./models/GRU.h5')
lstm_model = load_model('./models/LSTM.h5')
mlp_model = load_model('./models/MLP.h5')
rnn_model = load_model('./models/SimpleRNN.h5')

In [6]:
# Create a list of models
models = [ann_model, gru_model, lstm_model, mlp_model, rnn_model]

In [7]:
# Predict probabilities
y_pred_probs_ann = ann_model.predict(X)
y_pred_probs_gru = gru_model.predict(X[:, None, :])
y_pred_probs_lstm = lstm_model.predict(X[:, None, :])
y_pred_probs_mlp = mlp_model.predict(X)
y_pred_probs_rnn = rnn_model.predict(X[:, None, :])

29/29 [==============================] - 0s 2ms/step


In [8]:

# Convert probabilities to class predictions
y_pred_ann = np.round(y_pred_probs_ann).astype(int)
y_pred_gru = np.round(y_pred_probs_gru).astype(int)
y_pred_lstm = np.round(y_pred_probs_lstm).astype(int)
y_pred_mlp = np.argmax(y_pred_probs_mlp, axis=-1)
y_pred_rnn = np.round(y_pred_probs_rnn).astype(int)

In [9]:
# Ensure predictions are in the same shape
y_pred_ann = y_pred_ann.flatten()
y_pred_gru = y_pred_gru.flatten()
y_pred_lstm = y_pred_lstm.flatten()
y_pred_mlp = y_pred_mlp.flatten()
y_pred_rnn = y_pred_rnn.flatten()


In [96]:
def calculate_accuracy(weights, y_pred_ann, y_pred_gru, y_pred_lstm, y_pred_mlp, y_pred_rnn, y_true):
    # Calculate weighted predictions
    weighted_pred_ann = y_pred_ann * weights['ann']
    weighted_pred_gru = y_pred_gru * weights['gru']
    weighted_pred_lstm = y_pred_lstm * weights['lstm']
    weighted_pred_mlp = y_pred_mlp * weights['mlp']
    weighted_pred_rnn = y_pred_rnn * weights['rnn']
    
    # Sum the weighted predictions
    ensemble_prediction = (
        weighted_pred_ann +
        weighted_pred_gru +
        weighted_pred_lstm +
        weighted_pred_mlp +
        weighted_pred_rnn
    )

    # Convert probabilities to binary predictions (0 or 1)
    ensemble_prediction_binary = np.round(ensemble_prediction)

    # Calculate accuracy
    accuracy = np.mean(ensemble_prediction_binary == y_true)
    return accuracy



# Generate random weights
def generate_random_weights():
    weights = {
        'ann': np.random.rand(),
        'gru': np.random.rand(),
        'lstm': np.random.rand(),
        'mlp': np.random.rand(),
        'rnn': np.random.rand()
    }
    # Normalize weights to sum up to 1
    total_weight = sum(weights.values())
    for key in weights:
        weights[key] /= total_weight
    return weights


# Number of iterations for random weight generation
num_iterations = 1000

best_accuracy = 0
best_weights = None

# Iterate to find best weights
for _ in range(num_iterations):
    # Generate random weights
    weights = generate_random_weights()
    # Calculate accuracy for current weights
    accuracy = calculate_accuracy(weights, y_pred_ann, y_pred_gru, y_pred_lstm, y_pred_mlp, y_pred_rnn, y)
    # Check if current accuracy is the best so far
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_weights = weights

print("Best weights:", best_weights)
print("Best accuracy:", best_accuracy)

Best weights: {'ann': 0.4777893212349693, 'gru': 0.0079517789617278, 'lstm': 0.1848300137476593, 'mlp': 0.04912543384503784, 'rnn': 0.2803034522106058}
Best accuracy: 0.9455337690631809


>weights = {
    'ann': 0.3,
    'gru': 0.1,
    'lstm': 0.1,
    'mlp': 0.2,
    'rnn': 0.3
}
>Final Accuracy: 0.9433551198257081

Best weights: {'ann': 0.40443884470235036, 'gru': 0.006678696432566848, 'lstm': 0.1902283489898096, 'mlp': 0.286984900907338, 'rnn': 0.11166920896793521}
Best accuracy: 0.9455337690631809

>weights = {
    'ann': 0.4,
    'gru': 0.1,
    'lstm': 0.1,
    'mlp': 0.1,
    'rnn': 0.3
}
>Final Accuracy: 0.9455337690631809